In [1]:
#importing libraries for data ingestion and manipulation
import pandas as pd
import numpy as np
import plotly as px
import seaborn as sns
#Importing libraries for text analysis

import textstat
from tqdm import tqdm
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

import nltk
from nltk.corpus import stopwords
from collections import Counter
import re

nltk.download("stopwords")
stop_words = set(stopwords.words('english'))
tqdm.pandas()
nltk.download('punkt_tab')
#kmeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
vectorizer = TfidfVectorizer(stop_words='english')
from nltk import SnowballStemmer
param_stemmer = SnowballStemmer('english')
from nltk.tokenize import word_tokenize

from unidecode import unidecode
from sklearn import cluster

#For bar chart
import ast
import bar_chart_race as bcr
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.3 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
#read in csv data
info_df = pd.read_excel("csv_files/Presidential Debates Topic Analysis.xlsx",
                     sheet_name = "Sheet1")
headers = info_df.iloc[0]
info_df  = pd.DataFrame(info_df.values[1:], columns=headers)

In [ ]:
info_df.head()

,Election Year,Date,Type,Debate Name,URL,File name
0,2024,2024-10-01,General Election,Vice Presidential Debate in New York City,https://www.presidency.ucsb.edu/documents/vice...,debate_1.txt
1,2024,2024-09-10,General Election,"Presidential Debate in Philadelphia, Pennsylva...",https://www.presidency.ucsb.edu/documents/pres...,debate_2.txt
2,2024,2024-06-27,General Election,"Presidential Debate in Atlanta, Georgia (Biden...",https://www.presidency.ucsb.edu/documents/pres...,debate_3.txt
3,2024,2024-01-10,Primary Election: Republican Party,"Republican Candidates Debate in Des Moines, Iowa",https://www.presidency.ucsb.edu/documents/repu...,debate_4.txt
4,2024,2023-12-06,Primary Election: Republican Party,"Republican Candidates Debate in Tuscaloosa, Al...",https://www.presidency.ucsb.edu/documents/repu...,debate_5.txt


In [ ]:
def get_text(df):
    df = df.dropna(axis = 0)
    df = df.reset_index()
    df = df.drop(columns = ['index'])

    txt = []
    for i in tqdm(range(len(df))):
        #get file
        doc = df['File name'][i]
        test_doc = open('txt_files/'+doc,'r')

        #extract text
        text = test_doc.read()

        txt.append(text)
    df['Text'] = txt

    return df

In [ ]:
info_df = get_text(info_df)

100%|██████████| 175/175 [00:00<00:00, 3313.20it/s]


In [ ]:
def get_scores_2(df):

    #getting the sentiment analysis from Vader
    print("geting sentiment analysis ... ")
    df['Sentiment'] = df['Text'].progress_apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x))

    print(" ")
    #getting reading level from textstat
    print("Getting grade level ... ")
    df['Grade Level'] = df['Text'].progress_apply(lambda x: textstat.flesch_kincaid_grade(x))

    print(" ")
    #getting readability score also from text stat
    print("getting readability score ... ")
    df['Readability Score'] = df['Text'].progress_apply(lambda x: textstat.flesch_reading_ease(x))

    return df

In [ ]:
#info_df = get_scores_2(info_df)

In [ ]:
def get_text_blob(df):

    polarity = []
    subjectivity = []

    print('getting polarity and subjectivity ...')
    for i in tqdm(range(len(df))):
        text = df['Text'][i]

        blog = TextBlob(text)

        pol = blog.sentiment.polarity
        subj = blog.sentiment.subjectivity

        polarity.append(pol)
        subjectivity.append(subj)

    df['Text Polarity'] = polarity
    df['Text Subjectivity'] = subjectivity

    return df

In [ ]:
#info_df = get_text_blob(info_df)

In [ ]:
def extract_sentiment_scores(df):
    df['Negative Sentiment'] = df['Sentiment'].apply(lambda x: x['neg'])
    df['Neutral Sentiment'] = df['Sentiment'].apply(lambda x: x['neu'])
    df['Positive Sentiment'] = df['Sentiment'].apply(lambda x: x['pos'])

    return df

In [ ]:
info_df = extract_sentiment_scores(info_df)

In [4]:
#Save info df so you don't have to run each step again
# info_df.to_csv('csv_files/sentiment.csv',
#                index = True)

In [ ]:
words = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours',
'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself',
'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for',
'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to',
'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once',
'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very',
's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't",
'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',
 "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"}

In [ ]:
l2 = {'i','me','my','myself','we','our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves',
'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them',
'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those',
'am', 'is', 'are', 'was', 'were','be','been','being','have','has', 'had', 'having', 'do', 'does', 'did',
'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at',
'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before','after',
'above', 'below', 'to', 'from', 'up','down','in','out','on','off','over','under','again','further','then','once',
'here','there','when','where','why','how','all','any','both','each','few','more','most','other','some',
'such','no','nor','not','only','own','same','so','than','too','very','s','t','can','will','just','don','should',
'now',}

In [ ]:
irrelevant_words = {'trump', 'harris', 'biden', "president", 'govenor', 'haley', 'ramaswamy', 'yang', 'mr', 'vice', 'desantis',
'donald', 'kamala', 'joe', 'would', 'going', 'senator', 'welker', 'baier',
'pence', 'bloomberg', 'todd', 'thank', 'applause', 'clinton', 'hilary',
'kaine', 'sanders', 'think', 'cuomo', 'romney', 'obama', 'graham', 'blitzer',
'ryan', 'hemmer', 'moderator', 'king', 'mccain', 'john', 'wallace', 'cooper',
'bush', 'russert', 'smiley', 'stephanopoulos', 'think', 'want', 'koppel', 'bradley',
'thank', 'kemp', 'roberts', 'gore', 'reagan', 'ford',
                   "waltz", 'vance', 'muir', 'governor', 'christie', 'maccallum',
'albert', 'mayor', 'get', 'warren', 'klobuchar', 'buttigieg',
'donnell', 'steyer', 'way', 'let', 'said', 'one', 'done', 'way',
'look', 'said', 'get', 'lot', 'things', 'brennan', 'muir', 'let',
'davis', 'kelly', 'santorum', 'question', 'gibson', 'campaign', 'sen',
'edwards', 'richardson', 'said', 'take', 'new', 'matthews', 'get',
'bartiromo', 'ms', 'giuliani', 'one', 'go', 'need', 'gov', 'let',
'congressman', 'like', 'know',
                   'scott', 'holt', 'actually', 'us', 'also', "susan",
'vargas', 'nikki', 'thing', 'got', 'perino', 'varney',
'say', 'every', 'burgum', 'woodruff', 'well', 'sir',
'phillip', 'pfannenstiel', 'mitchell', 'lacey', 'sure',
'booker', 'lemon', 'bullock', 'guthrie', 'bennet',
'maddow', 'williamson', 'rourke', 'booker', 'castro',
'delaney', 'mosul', 'raddatz', 'well', 'hillary',
'quijano', 'holt', 'really', 'louis', 'also', 'rubio',
'marco', 'cruz', 'kasich', 'ramos', 'salinas', 'inaudible',
'secretary', 'ok', 'dickerson', 'carson', 'raddatz',
'much', 'talk', 'paul', 'huckabee', 'mike', 'fiorina',
'laughter','malley', 'andrea', 'smith', 'raddatz', 'pataki',
'jindal', 'pataki', 'quintanilla', 'chafee', 'anderson', 'webb',
'jake', 'hewitt', 'schieffer', 'crowley', '000', '2', 'lehrer',
'gingrich', 'newt', 'speaker', 'perry', 'sawyer', 'huntsman',
'bachmann', 'garrett', 'pelley', 'cain', '9', 'williams',
'pawlenty', 'brokaw', 'ifill', 'palin', 'lehrer', 'cheers',
'ronald', 'thompson', 'crosstalk', 'washburn', 'dodd',
                   'thompson', 'kucinich', 'williams', 'brownback', 'tancredo',
'rep', 'george', 'olbermann', 'wolf', 'goler', 'kerry', 'ifill',
'cheney', 'gwen', 'lehrer', 'brokaw', 'sharpton', 'tom', 'say',
'jennings', 'lieberman', 'distaso', 'dean', 'woodruff', 'shaw',
'keyes', 'alan', 'griffith', 'brown', 'bauer', 'ryerson',
'stanton', 'hatch', 'forbes', 'novak', 'q', 'cokie', 'dole',
'lehrer', '4', 'perot', 'simpson', 'quayle', 'bruno', 'stockdale',
'george', 'dukakis', 'put', 'bentsen', 'mondale', 'ferraro', 'vanocur',
'walters', 'smith', 'carter', 'jimmy', 'moyers', 'could', '2', 'uh',
'hoge', 'two', 'kennedy', 'nixon', 'howe', 'shadel', 'three', 'mcgee',
   'walz', 'margaret', 'tim','donnel',

                   }


In [ ]:
other_words = irrelevant_words.union(l2).union(words)

In [ ]:
#removes a list of words (ie. stopwords) from a tokenized list.
def removeWords(listOfTokens, listOfWords):
    return [token for token in listOfTokens if token not in listOfWords]

# applies stemming to a list of tokenized words
def applyStemming(listOfTokens, stemmer):
    return [stemmer.stem(token) for token in listOfTokens]

# removes any words composed of less than 2 or more than 21 letters
def twoLetters(listOfTokens):
    twoLetterWord = []
    for token in listOfTokens:
        if len(token) <= 2 or len(token) >= 21:
            twoLetterWord.append(token)
    return twoLetterWord

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#cleaning text
def processCorpus(df):

    clean_text = []

    for i in tqdm(range(len(df))):

        text = df['Text'][i]

        text = text.replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
        text = text.replace(',', '')          # Removes commas
        text = text.rstrip('\n')              # Removes line breaks
        text = text.casefold()                # Makes all letters lowercase

        text = re.sub('\W_',' ', text)        # removes specials characters and leaves only words
        text = re.sub("\S*\d\S*"," ", text)   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
        text = re.sub("\S*@\S*\s?"," ", text) # removes emails and mentions (words with @)
        text = re.sub(r'http\S+', '', text)   # removes URLs with http
        text = re.sub(r'www\S+', '', text)    # removes URLs with www

        listOfTokens = word_tokenize(text)
        twoLetterWord = twoLetters(listOfTokens)

        listOfTokens = removeWords(listOfTokens, stop_words)
        listOfTokens = removeWords(listOfTokens, twoLetterWord)

        listOfTokens = removeWords(listOfTokens, other_words)

        listOfTokens = applyStemming(listOfTokens, param_stemmer)
        listOfTokens = removeWords(listOfTokens, other_words)

        text   = " ".join(listOfTokens)
        text = unidecode(text)

        clean_text.append(text)
    df['Clean Text'] = clean_text
    return df

In [ ]:
sentiment_2 = processCorpus(sentiment_df)

100%|██████████| 175/175 [02:37<00:00,  1.11it/s]


In [ ]:
def get_stat_weight(df):
    weight_array = []
    for i in tqdm(range(len(df))):
        # Get the 'Clean Text' for the current row
        corpus = df['Clean Text'][i]

        if isinstance(corpus, str):
            corpus = [corpus]
        # Create a TfidfVectorizer and fit_transform on the corpus
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(corpus)

        tf_idf = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names_out())
        # Get the top 20 words by TF-IDF score for the current row
        weight_array.append(tf_idf.T.nlargest(20, 0))

    df['Weight'] = weight_array
    # Return the DataFrame with the 'Weight' column added
    return df

In [ ]:
sentiment3 = get_stat_weight(sentiment_2)

100%|██████████| 175/175 [00:01<00:00, 112.27it/s]


In [ ]:
sentiment3 = sentiment3.drop(columns = ['Unnamed: 0'])

In [ ]:
def get_common_words(df):
    exclude = stop_words.union(other_words)
    common = []

    for i in tqdm(range(len(df))):
        d_text = df['Text'][i]
        text = d_text.lower()
        words = re.findall(r'\w+', text)
        filtered_words =[word for word in words if word not in exclude]

        word_counts = Counter(filtered_words)
        top_20 = word_counts.most_common(25)

        common.append(top_20)

    df['Common Words'] = common

    return df

In [ ]:
sentiment3 = get_common_words(sentiment3)

100%|██████████| 175/175 [00:02<00:00, 77.98it/s] 


In [ ]:
sentiment3['Common Words'][0]

[('people', 83),
 ('make', 67),
 ('american', 59),
 ('country', 51),
 ('first', 47),
 ('time', 42),
 ('issue', 38),
 ('right', 35),
 ('care', 34),
 ('back', 33),
 ('housing', 32),
 ('minnesota', 30),
 ('americans', 30),
 ('years', 30),
 ('home', 29),
 ('able', 29),
 ('family', 28),
 ('debate', 26),
 ('seen', 26),
 ('federal', 26),
 ('made', 24),
 ('states', 24),
 ('folks', 24),
 ('border', 24),
 ('pay', 23)]

In [ ]:
sentiment4 = sentiment3.drop(columns = ['Text',
                                        'Sentiment',
                                        'Clean Text',
                                        'Cluster'])

In [ ]:
sentiment4.to_csv('csv_files/output.csv')

In [ ]:
sentiment5 = pd.read_excel('csv_files/Presidential Debate Text Analysis.xlsx',
                           sheet_name = 'output')

In [ ]:
general = sentiment5[sentiment5['Type'] == 'General Election']

In [ ]:
import ast
all_words = {}

for index, row in general.iterrows():
    date = row['Date']
    words_list = ast.literal_eval(row['Common Words'])
    words_dict = dict(words_list)  # Now convert the list of tuples to a dictionary
    all_words[date] = words_dict

# Convert this into a DataFrame, with dates as the index and words as columns
word_df = pd.DataFrame(all_words).T.fillna(0)

# Ensure the DataFrame is sorted by date
word_df = word_df.sort_index()


In [ ]:
word_df.head()

,people,make,american,country,first,time,issue,right,care,back,...,far,cuba,meet,conference,regrets,teachers,salaries,growth,eight,power
1960-09-26,32.0,0.0,17.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,24.0,23.0,22.0,18.0,17.0
1960-10-07,47.0,24.0,19.0,27.0,15.0,23.0,0.0,0.0,0.0,0.0,...,20.0,17.0,16.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0
1960-10-13,0.0,23.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1960-10-21,36.0,20.0,28.0,21.0,37.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1976-09-23,77.0,0.0,0.0,47.0,24.0,31.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:

# Saving the bar chart race as an MP4 video
bcr.bar_chart_race(
    df=word_df,
    filename='political_debates_3.mp4',  # Save as MP4
    title='Common Words in General Election Debates Over Time',
    n_bars=7,  # Number of bars (words) to display at a time
    period_length=5000,
    steps_per_period=1000,
    interpolate_period=True
)



In [15]:
print("We done")

We done




---



In [20]:
word_df_accumulated = word_df.copy().reset_index()

# Reset the index to access 'Date' as a regular column
word_df_accumulated = word_df_accumulated.rename(columns={'index': 'Date'})

# Iterate through each row (debate) and accumulate word counts
for i in range(1, len(word_df_accumulated)):
    # Get previous and current dates
    prev_date = word_df_accumulated.loc[i - 1, 'Date']
    current_date = word_df_accumulated.loc[i, 'Date']

    # Get previous and current word counts as dictionaries
    prev_words = word_df_accumulated.loc[i - 1].drop('Date').to_dict()
    current_words = word_df_accumulated.loc[i].drop('Date').to_dict()

    # Accumulate counts for words that appeared before and are mentioned again
    for word, count in current_words.items():
        if word in prev_words and prev_words[word] > 0:  # Check if word existed before
            current_words[word] += prev_words[word]  # Accumulate the count

    # Update the current row with the accumulated word counts
    # Use word_df_accumulated.columns != 'Date' to select all columns except 'Date'
    word_df_accumulated.loc[i, word_df_accumulated.columns != 'Date'] = pd.Series(current_words)

# Set 'Date' back as the index
word_df_accumulated.set_index('Date', inplace=True)

# Now generate the bar chart race using the accumulated word counts
bcr.bar_chart_race(
    df=word_df_accumulated,  # Use the accumulated DataFrame
    filename='ge_accumulation_2.mp4',  # Save to file
    title='Accumulated Common Words in General Electioin Debates Over Time',
    n_bars=7,
    period_length=1000,
    steps_per_period=200,
    interpolate_period=True
)